In [ ]:
import numpy as np
import math
from matplotlib import pyplot as plt
from skimage import io, util
import os

os.chdir(os.path.dirname(__file__))


In [ ]:
filename_Grayimage = 'PeppersBayerGray.bmp'
filename_RGB = 'PeppersRGB.bmp'
filename_gridB = 'gridB.bmp'
filename_gridR = 'gridR.bmp'
filename_gridG = 'gridG.bmp'

In [ ]:
# part I

img = io.imread(filename_Grayimage, as_gray=True)
og = io.imread(filename_RGB)

h, w = img.shape

# our final image will be a 3 dimensional image with 3 channels
rgb = np.zeros((h, w, 3), np.uint8)

# reconstruction of the green channel IG
IG = np.copy(img) # copy the image into each channel

for row in range(0, h, 4): # loop step is 4 since our mask size is 4.
    for col in range(0, w, 4): # loop step is 4 since our mask size is 4.
        
        IG[row, col + 1] = (int(img[row, col]) + int(img[row, col + 2])) / 2 # B
        IG[row, col + 3] = (int(img[row, col + 2]) + int(img[row + 1, col + 3])) / 2 # D

        IG[row + 1, col] = (int(img[row, col]) + int(img[row + 2, col])) / 2 # E
        IG[row + 1, col + 2] = (int(img[row + 1, col + 1]) + int(img[row + 1, col + 3]) + int(img[row, col + 2]) + int(img[row + 2, col + 2])) / 4 # G

        IG[row + 2, col + 1] = (int(img[row + 2, col]) + int(img[row + 2, col + 2]) + int(img[row + 1, col + 1]) + int(img[row + 3, col + 1])) / 4 # J
        IG[row + 2, col + 3] = (int(img[row + 1, col + 3]) + int(img[row + 3, col + 3])) / 2 # L

        IG[row + 3, col] = (int(img[row + 2, col]) + int(img[row + 3, col + 1])) / 2 # M
        IG[row + 3, col + 2] = (int(img[row + 3, col + 1]) + int(img[row + 3, col + 3])) / 2 # O
        

# reconstruction of the red channel IR
IR = np.copy(img)

for row in range(0, h, 4): # loop step is 4 since our mask size is 4.
    for col in range(0, w, 4): # loop step is 4 since our mask size is 4.

        IR[row, col + 2] = (int(img[row, col + 1]) + int(img[row, col + 3])) / 2 # C
        IR[row + 1, col + 1] = (int(img[row, col + 1]) + int(img[row + 2, col + 1])) / 2 # F
        IR[row + 1, col + 2] = (int(img[row, col + 1]) + int(img[row, col + 3]) + int(img[row + 2, col + 1]) + int(img[row + 2, col + 3])) / 4 # G
        IR[row, col + 2] = (int(img[row, col + 3]) + int(img[row + 2, col + 3])) / 2 # H
        IR[row, col + 2] = (int(img[row + 2, col + 1]) + int(img[row + 2, col + 3])) / 2 # K
        IR[:, col] = IR[:, col + 1]
        IR[row + 3, :] = IR[row + 2, :]
        
# reconstruction of the blue channel IB
IB = np.copy(img)

for row in range(0, h, 4): # loop step is 4 since our mask size is 4.
    for col in range(0, w, 4): # loop step is 4 since our mask size is 4.

        IB[row + 1, col + 1] = (int(img[row + 1, col]) + int(img[row + 1, col + 2])) / 2 # F
        IB[row + 2, col] = (int(img[row + 1, col]) + int(img[row + 3, col])) / 2 # I
        IB[row + 1, col + 2] = (int(img[row + 1, col]) + int(img[row + 1, col + 2]) + int(img[row + 3, col]) + int(img[row + 3, col + 2])) / 4 # J
        IB[row, col + 2] = (int(img[row + 1, col + 2]) + int(img[row + 3, col + 2])) / 2 # K
        IB[row, col + 2] = (int(img[row + 3, col]) + int(img[row + 3, col + 2])) / 2 # N
        IB[row, :] = IB[row + 1, :]
        IB[:, col + 3] = IB[:, col + 2]

# merge the channels
rgb[:, :, 0] = IR
rgb[:, :, 1] = IG
rgb[:, :, 2] = IB

diff = util.compare_images(og, rgb)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 20))
axes[0].imshow(og)
axes[0].set_title("Original")

axes[1].imshow(rgb)
axes[1].set_title("RGB")

axes[2].imshow(diff)
axes[2].set_title("Diff")

plt.tight_layout()
plt.show()